In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
trash_df = pd.read_csv("C:/Users/dmora/Documents/NSS/python/projects/trash-haulers-badgers/data/trash_hauler_report_ll.csv")
trash_df

In [ ]:
trash_df = trash_df.rename(columns={'Request ': 'Request'})

In [ ]:
trash_df.sort_values('Request Number', ascending=True)

In [ ]:
trash_df['Trash Hauler'] = trash_df['Trash Hauler'].str.upper()

In [ ]:
trash_df = trash_df.drop(columns = ['State Plan X', 'State Plan Y'])

In [ ]:
trash_df = trash_df.rename(columns={'Request Number': 'Request_Number'})

In [ ]:
trash_df = trash_df.rename(columns={'Incident Address': 'Incident_Address'})

In [ ]:
trash_df = trash_df.rename(columns={'Zip Code': 'Zip_Code'})

In [ ]:
trash_df = trash_df.rename(columns={'Trash Hauler': 'Trash_Hauler'})

In [ ]:
trash_df = trash_df.rename(columns={'Trash Route': 'Trash_Route'})

In [ ]:
trash_df = trash_df.rename(columns={'Council District': 'Council_District'})

In [ ]:
trash_df = trash_df.fillna(0)
trash_df['Council_District'] = trash_df.Council_District.astype(int)

In [ ]:
trash_df['Zip_Code'] = trash_df.Zip_Code.astype(int)

In [ ]:
trash_df['Description'] = trash_df['Description'].str.lower()

In [ ]:
trash_df.loc[trash_df['Description'].isin(['miss', 'missed', 'not picked up', 'no pickup'])]

In [ ]:
trash_dff = trash_df[trash_df['Description'].str.contains('miss|missed|no pickup|not picked up|no trash pickup|not been picked up|skipped|never gets picked up|not being picked up|not collected|failure to pick|waiting for pick|did not get picked|skip|has not been|did not pick|not pick up|not pickup|did not empty|pass|no pick|no trash pick|been picked up|not getting picked|not had trash pick|didnt get picked|cart still out|cart been out|failed to pick up|never came|still on curb|still full|never picked up|cart has been out|not emptied|carts are still out|arts still out|still out', na = False, case = False)]
trash_dff

In [ ]:
red_river = trash_dff[trash_dff['Trash_Hauler'] == 'RED RIVER']
waste_indus = trash_dff[trash_dff['Trash_Hauler'] == 'WASTE IND']
metro = trash_dff[trash_dff['Trash_Hauler'] == 'METRO']
hauler_unk = trash_dff[trash_dff['Trash_Hauler'] == 0]

In [ ]:
red_river.reset_index(drop = True)

In [ ]:
red_river['Incident_Address'].str.lower()

In [ ]:
red_river.sort_values(['Date Opened', 'Incident_Address'])

In [ ]:
red_river = red_river.rename(columns={'Date Opened': 'Date_Opened'})

In [ ]:
red_river.sort_values(['Date_Opened', 'Incident_Address'])

In [ ]:
red_river['Date_Opened'] = pd.to_datetime(red_river['Date_Opened'])

In [ ]:
red_river.sort_values(['Date_Opened', 'Incident_Address'])

In [ ]:
addy_count = red_river.Incident_Address.value_counts()>=3

In [ ]:
addy_count.to_frame()

In [ ]:
red_river = pd.merge(red_river, addy_count, left_on = 'Incident_Address', right_on = 'Incident_Address')

In [ ]:
red_river_three = red_river.loc[red_river['count']== True]

In [ ]:
red_river_three = red_river_three.reset_index(drop = True)

In [ ]:
red_river_three.head(10)

In [ ]:
red_river_three.Date_Opened = pd.to_datetime(red_river_three.Date_Opened, format='%d/%m/%Y')
g = pd.Grouper(key='Date_Opened', freq='6MS')
red_river_occ = red_river_three.assign(occ=red_river_three.groupby(['Incident_Address', g]).cumcount())

In [ ]:
red_river_occ.head(10)

In [ ]:
for index, row in red_river_occ.iterrows():
    if row.occ == 0:
        red_river_occ.loc[index, 'fine'] = 0
    elif row.occ == 1:
        red_river_occ.loc[index, 'fine'] = 0
    elif row.occ == 2:
        red_river_occ.loc[index, 'fine'] = 1500
    else:
         red_river_occ.loc[index, 'fine'] = 0

In [ ]:
total_red_river = red_river_occ['fine'].sum()
print(total_red_river)

In [ ]:
waste_indus

In [ ]:
waste_indus = waste_indus.reset_index(drop = True)

In [ ]:
waste_indus['Incident_Address']  = waste_indus['Incident_Address'].str.lower()
waste_indus

In [ ]:
waste_indus['Incident_Address'] = waste_indus['Incident_Address'].str.split(',', expand = True)[0]
waste_indus

In [ ]:
waste_indus.drop(['Council_District'], axis = 1)

In [ ]:
waste_indus = waste_indus.rename(columns={'Date Opened': 'Date_Opened'})
waste_indus

In [ ]:
waste_indus.sort_values(by=['Date_Opened', 'Incident_Address'])

In [ ]:
addy_count_wi = waste_indus.Incident_Address.value_counts()>=3

In [ ]:
addy_count_wi.to_frame()

In [ ]:
waste_indus = pd.merge(waste_indus, addy_count_wi, left_on = 'Incident_Address', right_on = 'Incident_Address')
waste_indus.head(20)

In [ ]:
waste_indus_three = waste_indus.loc[waste_indus['count'] == True]

In [ ]:
waste_indus_three = waste_indus_three.reset_index(drop = True)
waste_indus_three

In [ ]:
waste_indus_three['Date_Opened'] = pd.to_datetime(waste_indus_three['Date_Opened'])

In [ ]:
waste_indus_three.Date_Opened = pd.to_datetime(waste_indus_three.Date_Opened, format='%d/%m/%Y')
g = pd.Grouper(key='Date_Opened', freq='6MS')
waste_indus_occ = waste_indus_three.assign(occ=waste_indus_three.groupby(['Incident_Address', g]).cumcount())

In [ ]:
waste_indus_occ = waste_indus_occ.drop(['Council_District'], axis = 1)

In [ ]:
waste_indus_occ

In [ ]:
for index, row in waste_indus_occ.iterrows():
    if row.occ == 0:
        waste_indus_occ.loc[index, 'fine'] = 0
    elif row.occ == 1:
        waste_indus_occ.loc[index, 'fine'] = 0
    elif row.occ == 2:
        waste_indus_occ.loc[index, 'fine'] = 1500
    else:
         waste_indus_occ.loc[index, 'fine'] = 0

In [ ]:
waste_indus_occ

In [ ]:
total_waste_indus = waste_indus_occ['fine'].sum()
print(total_waste_indus)

In [ ]:
metro = metro.reset_index(drop = True)
metro

In [ ]:
metro = metro.rename(columns={'Date Opened': 'Date_Opened'})
metro

In [ ]:
metro.sort_values(by=['Date_Opened', 'Incident_Address'])

In [ ]:
addy_count_met = metro.Incident_Address.value_counts()>=3

In [ ]:
addy_count_met.to_frame()

In [ ]:
metro = pd.merge(metro, addy_count_met, left_on = 'Incident_Address', right_on = 'Incident_Address')
metro.head(20)

In [ ]:
metro_three = metro.loc[metro['count'] == True]

In [ ]:
metro_three = metro_three.reset_index(drop = True)
metro_three

In [ ]:
metro_three['Date_Opened'] = pd.to_datetime(metro_three['Date_Opened'])

In [ ]:
metro_three.Date_Opened = pd.to_datetime(metro_three.Date_Opened, format='%d/%m/%Y')
g = pd.Grouper(key='Date_Opened', freq='6MS')
metro_occ = metro_three.assign(occ=metro_three.groupby(['Incident_Address', g]).cumcount())

In [ ]:
metro_occ = metro_occ.drop(['Council_District'], axis = 1)

In [ ]:
for index, row in metro_occ.iterrows():
    if row.occ == 0:
        metro_occ.loc[index, 'fine'] = 0
    elif row.occ == 1:
        metro_occ.loc[index, 'fine'] = 0
    elif row.occ == 2:
        metro_occ.loc[index, 'fine'] = 1500
    else:
        metro_occ.loc[index, 'fine'] = 0

In [ ]:
total_metro = metro_occ['fine'].sum()
print(total_metro)

In [ ]:
print(total_red_river)

In [ ]:
print(total_waste_indus)

Red River: 14,395 total complaints; 11,568 missed pickups; $639,000 fines owed; 73 percent of total missed complaints; FIND fines owed / total missed pickups = cost per miss; FIND missed complaints / total RR complaints = missed complaint percent overall
Waste Industries: 1,350 total complaints; 994 missed pickups; $52,500 fines owed; 6 percent of total missed complaints; FIND fines owed / total missed pickups = cost per miss; FIND missed complaints / total RR complaints = missed complaint percent overall
Metro: 3,580 total complaints; 2,669 missed pickups; $124,500 fines owed; 17 percent of total missed complaints; FIND fines owed / total missed pickups = cost per miss; FIND missed complaints / total RR complaints = missed complaint percent overall
Unkown: 901 total complaints, 4 percent of total missed complaints; FIND fines owed / total missed pickups = cost per miss; FIND missed complaints / total RR complaints = missed complaint percent overall

In [ ]:
row_count_rr = len(red_river)
row_count_rr

In [ ]:
row_count_wi = len(waste_indus)
row_count_wi

In [ ]:
row_count_met = len(metro)
row_count_met

In [ ]:
row_count_unk = len(hauler_unk)
row_count_unk

In [ ]:
row_count_total = len(trash_dff)
row_count_total

In [ ]:
total_missed = row_count_rr + row_count_wi + row_count_met + row_count_unk
total_missed

In [ ]:
rr_miss_perc = row_count_rr/total_missed
rr_miss_perc

In [ ]:
wi_miss_perc = row_count_wi/total_missed
wi_miss_perc

In [ ]:
unk_miss_perc = row_count_unk/total_missed
unk_miss_perc

In [ ]:
rr_miss_percentage = f"{rr_miss_perc:.00%}"
rr_miss_percentage

In [ ]:
wi_miss_percentage = f"{wi_miss_perc:.00%}"
wi_miss_percentage

In [ ]:
met_miss_percentage = f"{met_miss_perc:.00%}"
met_miss_percentage

In [ ]:
unk_miss_percentage = f"{unk_miss_perc:.00%}"
unk_miss_percentage

In [ ]:
red_river_three.Date_Opened = pd.to_datetime(red_river_three.Date_Opened, format='%d/%m/%Y')
g = pd.Grouper(key='Date_Opened', freq='6MS')
red_river_roll = red_river_three.assign(occ=red_river_three.groupby(['Incident_Address', g]).cumcount())

In [ ]:
red_river_roll

In [ ]:
red_river_roll_occ = red_river_roll.occ.rolling(3, win_type ='triang').sum()
red_river_roll["roll"] = red_river_roll_occ

In [ ]:
red_river_roll.head(10)

In [ ]:
red_river_three.Date_Opened = pd.to_datetime(red_river_three.Date_Opened, format='%d/%m/%Y')
g = pd.Grouper(key='Date_Opened', freq='6MS')
red_river_occ_roll = red_river_three.assign(occ=red_river_three.groupby(['Incident_Address', g]).cumcount())
red_river_occ_roll.head(8)

In [ ]:
red_river_three.reset_index(drop = True)

In [ ]:
for index, row in red_river_occ.iterrows():
    if row.occ == 0:
        red_river_occ.loc[index, 'fine'] = 0
    elif row.occ == 1:
        red_river_occ.loc[index, 'fine'] = 0
    elif row.occ == 2:
        red_river_occ.loc[index, 'fine'] = 1500
    else:
         red_river_occ.loc[index, 'fine'] = 0